<a href="https://colab.research.google.com/github/matgonzalez23/MetNumUN2022I/blob/main/Lab_9/Week4IterativeMethodsForLinearSystemsGroup7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



<h1>Grupo 1</h1>
<ul>
<li> <h3>Santiago Alvarez Ricardo</h3> </li>
<li> <h3>Juan Sebastian Torres</h3> </li>
<li> <h3>Juan Sebastian Suarez</h3> </li>



# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(7860)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.3412295173697629

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  0.00000000e+00,  5.55111512e-17,  0.00000000e+00,
       -1.38777878e-17,  0.00000000e+00,  0.00000000e+00, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [7]:
def Jacobi_iteration(A, b, eps = 1e-7, n_iter = 50):
    
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b
    
    x = np.ones(n)
    for _ in range(n_iter):
        x = BB @ x + c
    return x

In [8]:
x = Jacobi_iteration(A, b)
print(x)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

[ 0.04709667  0.04637916  0.01896904  0.02649428 -0.0072363   0.04783264
  0.05148203  0.0480632   0.04103157 -0.00139149]


In [9]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(Jacobi_iteration(A1, b)-np.linalg.solve(A1, b)))
    

0.3649436955565377 2.5272843339497796e-17
0.392202757062319 1.4726004136889805e-17
0.4238664775089971 3.440048942762026e-17
0.46109741381493485 3.2650150334626576e-17
0.5055072154826658 4.2571471319696836e-17
0.5593969721171765 1.297800416734946e-16
0.6261706848142661 2.9826139540701476e-14
0.7110875006701638 1.698052662699032e-11
0.8227286178703906 2.4426088884358968e-08
0.9761229877685789 0.00012224667098061605
1.2002403115370013 3.5845739482506396
1.559193183042101 1576734.3508510473
2.2297334447100097 75912927672484.08
3.956249174421963 1.1348387282713009e+26
24.799704005206664 5.702468898356118e+61


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [10]:
def seidel_iteration(A, b, eps = 1e-7, n_iter = 50):
    x = np.ones(b.shape[0])
    
    for _ in range(n_iter):
        for k in range(b.shape[0]):
              x[k] = (b[k]-np.dot(A[k][:k], x[:k])- np.dot(A[k][k+1:], x[k+1:]))/A[k,k]
    return x

In [11]:
x = seidel_iteration(A,b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [12]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(seidel_iteration(A1, b)-np.linalg.solve(A1, b)))

0.3649436955565377 2.7097246932071044e-17
0.392202757062319 1.1317325853115499e-17
0.4238664775089971 2.893662274657725e-17
0.46109741381493485 3.2139234526580756e-17
0.5055072154826658 4.6805398318361535e-17
0.5593969721171765 4.473434455583162e-17
0.6261706848142661 2.8025315438821013e-17
0.7110875006701638 4.4565853017184997e-17
0.8227286178703906 5.679729445007588e-17
0.9761229877685789 4.2712612100832585e-17
1.2002403115370013 1.0049799730474779e-16
1.559193183042101 1.4033044459902308e-16
2.2297334447100097 1.8729795718427684e-16
3.956249174421963 0.00024282655825824537
24.799704005206664 1.2252142582332306e+126


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [13]:
def minimum_res_scheme(A, b, eps = 1e-7, n_iter = 50):
    x = np.ones(b.shape[0])
    
    for _ in range(n_iter):
        r = A @ x - b
        k = (r @ A @ r)/np.linalg.norm(A @ r)**2
        x = x - k*r
        
    return x

In [14]:
x = minimum_res_scheme(A,b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [15]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(minimum_res_scheme(A1, b)-np.linalg.solve(A1, b)))

0.3649436955565377 nan
0.392202757062319 1.2695817633565476e-17
0.4238664775089971 3.929062510380681e-17
0.46109741381493485 3.2788109330792944e-17
0.5055072154826658 2.949029909160572e-17
0.5593969721171765 3.82033216616167e-17
0.6261706848142661 4.45996022480856e-17
0.7110875006701638 5.605063087764203e-17
0.8227286178703906 3.756787906405246e-17
0.9761229877685789 nan
1.2002403115370013 9.20711736692975e-17
1.559193183042101 7.52414185906026e-16
2.2297334447100097 1.965518723746668e-11
3.956249174421963 0.0007253197462527024
24.799704005206664 43.57313797883575


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
